In [2]:
# !pip install detoxify

In [ ]:
from detoxify import Detoxify

d = Detoxify("original")

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
100%|██████████| 418M/418M [00:03<00:00, 114MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
d = Detoxify("original", device="cuda")

In [ ]:
res = d.predict("bitch lasagna")
toxic = bool(res["toxicity"] > 0.5)
toxic, res

(True,
 {'toxicity': np.float32(0.98933583),
  'severe_toxicity': np.float32(0.10260555),
  'obscene': np.float32(0.95540065),
  'threat': np.float32(0.0013866336),
  'insult': np.float32(0.9023501),
  'identity_attack': np.float32(0.017322378)})

In [ ]:
import pandas as pd

df = pd.read_csv("all_transcriptions.csv").fillna("")

In [ ]:
df["detoxify_results"] = None
df["detoxify_toxic"] = None

for idx, row in df.iterrows():
  print(idx, end="-")
  res = d.predict(row["Transcription"])
  toxic = bool(res["toxicity"] > 0.5)
  df.at[idx, "detoxify_toxic"] = toxic
  df.at[idx, "detoxify_results"] = res

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-103-104-105-106-107-108-109-110-111-112-113-114-115-116-117-118-119-120-121-122-123-124-125-126-127-128-129-130-131-132-133-134-135-136-137-138-139-140-141-142-143-144-145-146-147-148-149-150-151-152-153-154-155-156-157-158-159-160-161-162-163-164-165-166-167-168-169-170-171-172-173-174-175-176-177-178-179-180-181-182-183-184-185-186-187-188-189-190-191-192-193-194-195-196-197-198-199-200-201-202-203-204-205-206-207-208-209-210-211-212-213-214-215-216-217-218-219-220-221-222-223-224-225-226-227-228-229-230-231-232-233-234-235-236-237-238-239-240-241-242-243-244-245-246-247-248-249-250-251-252-253-254-255-256-257-258-259-260-261-262-263-264-265-266-267-268-269-270-271-272-273-274-275-276-27

In [24]:
df.head()

,Transcription,label,detoxify_results,detoxify_toxic
0,the representatives of the united states of th...,nonhate,"{'toxicity': 0.00058094965, 'severe_toxicity':...",False
1,it's a queer little cake and a cup of butter a...,nonhate,"{'toxicity': 0.025781365, 'severe_toxicity': 0...",False
2,one somebody who actually could be social when...,nonhate,"{'toxicity': 0.058489792, 'severe_toxicity': 0...",False
3,the services i had to have is in a you said mo...,nonhate,"{'toxicity': 0.00078568317, 'severe_toxicity':...",False
4,he added and he added i heard it and i heard t...,nonhate,"{'toxicity': 0.01859717, 'severe_toxicity': 0....",False


In [ ]:
import time

import openai
import pandas as pd

openai.api_key = ""


def moderate_df(df, text_column="Transcription", batch_size=32):

    for i in range(0, len(df), batch_size):
        print(i, end="-")
        batch = df.iloc[i : i + batch_size]
        texts = batch[text_column].tolist()

        try:
            response = openai.moderations.create(input=texts)
            results = response.model_dump()["results"]
            hateflags = [i["categories"]["hate"] for i in results]

            for idx, flag in zip(batch.index, hateflags):
                df.at[idx, "openai_hate"] = flag

            for idx, result in zip(batch.index, results):
                df.at[idx, "openai_results"] = result

            time.sleep(2)

        except Exception as e:
            print(e)
            time.sleep(10)
            continue
    return df

In [26]:
mdf = moderate_df(df, text_column="Transcription", batch_size=32)

0-Incompatible indexer with Series
32-64-96-128-160-192-224-256-288-320-352-384-416-448-480-512-544-576-608-640-672-704-736-768-800-832-864-896-928-960-992-1024-1056-1088-1120-1152-1184-1216-1248-1280-1312-1344-1376-1408-1440-1472-1504-1536-1568-1600-1632-1664-1696-1728-1760-1792-1824-1856-1888-1920-1952-1984-

In [29]:
mdf.head()

,Transcription,label,detoxify_results,detoxify_toxic,openai_hate,openai_results
0,the representatives of the united states of th...,nonhate,"{'toxicity': 0.00058094965, 'severe_toxicity':...",False,False,NaN
1,it's a queer little cake and a cup of butter a...,nonhate,"{'toxicity': 0.025781365, 'severe_toxicity': 0...",False,False,NaN
2,one somebody who actually could be social when...,nonhate,"{'toxicity': 0.058489792, 'severe_toxicity': 0...",False,False,NaN
3,the services i had to have is in a you said mo...,nonhate,"{'toxicity': 0.00078568317, 'severe_toxicity':...",False,False,NaN
4,he added and he added i heard it and i heard t...,nonhate,"{'toxicity': 0.01859717, 'severe_toxicity': 0....",False,False,NaN


In [ ]:
mdf[mdf["label"] == "nonhate"]["detoxify_toxic"].value_counts()

,count
detoxify_toxic,
False,988
True,12


In [ ]:
mdf[mdf["label"] != "nonhate"]["detoxify_toxic"].value_counts()

,count
detoxify_toxic,
False,934
True,75


### Detoxify  
| **prediction \ actual** | **Nonhate** (1000) | **Hate** (1009) | **Total** |
| ---------------------- | ------------------ | --------------- | --------- |
| **Non-toxic**           | 988                | 934             | 1922      |
| **Toxic**               | 12                 | 75              | 87        |
| **Total**               | 1000               | 1009            | 2009      |

Accuracy= (988+75)/2009 ≈ 52.96%

In [ ]:
mdf[mdf["label"] == "nonhate"]["openai_hate"].value_counts()

,count
openai_hate,
False,1000


In [ ]:
mdf[mdf["label"] != "nonhate"]["openai_hate"].value_counts()

,count
openai_hate,
False,983
True,26


### OpenAI Moderation API
| **prediction \ actual** | **Nonhate** (1000) | **Hate** (1009) | **Total** |
| ---------------------- | ------------------ | --------------- | --------- |
| **Not Hate (False)**    | 1000               | 983             | 1983      |
| **Hate (True)**         | 0                  | 26              | 26        |
| **Total**               | 1000               | 1009            | 2009      |


Accuracy = (1000 + 26) / 2009 = 51.07%